Nos basamos en el siguiente paper https://arxiv.org/pdf/2202.00599.pdf

In [8]:
#Usar Sklearn para minmax
import sklearn as sk
from sklearn.preprocessing import MinMaxScaler

# Pandas y numpy
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pylab import rcParams
from datetime import datetime

# Qiskit
import qiskit
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit.circuit import library 
from qiskit import Aer, execute
from qiskit.circuit import ParameterVector
import math
import random

from qiskit.quantum_info.operators import Operator, Pauli

# Minimizar
from scipy.optimize import minimize

np.random.seed(1)

In [55]:
df=pd.read_csv('./datos/AirPassengers.csv')
df['Month']=pd.to_datetime(df['Month'], infer_datetime_format=True)
df=df.set_index(['Month'])
df_log=np.log(df)
data_shift=df_log-df_log.shift()
data_shift = data_shift.dropna()

# Crear los conjuntos
ptrain=0.8
pasajeros = np.array(data_shift['#Passengers'])
pasajeros_train = pasajeros[:int(pasajeros.shape[0]*0.8)]
pasajeros_test = pasajeros[int(pasajeros.shape[0]*0.8):]

tiempo = data_shift.drop(['#Passengers'], axis=1)
tiempo_train = tiempo.head(int(pasajeros.shape[0]*0.8))
tiempo_test = tiempo.tail(pasajeros.shape[0] - int(pasajeros.shape[0]*0.8))

# Pasemos a numpy
window = 2;
tam_train = pasajeros_train.shape[0] - window
tam_test = pasajeros_test.shape[0] - window
conjunto_train = []
objetivo_train = []
conjunto_test = []
objetivo_test = []

for i in range(0,tam_train):
    conjunto_train.append(pasajeros_train[0+i:window+i])
    objetivo_train.append(pasajeros_train[window+i])

for i in range(0,tam_test):
    conjunto_test.append(pasajeros_test[0+i:window+i])
    objetivo_test.append(pasajeros_test[window+i])
    
objetivo_train = np.array(objetivo_train)
conjunto_train = np.array(conjunto_train)

#No vamos a usar ahora mismo ningún conjunto de validación...

objetivo_test = np.array(objetivo_test)
conjunto_test = np.array(conjunto_test)

array([-0.02298952, -0.06402186,  0.10948423,  0.0919375 ,  0.        ,
       -0.08455739, -0.13353139, -0.13473259,  0.12629373, -0.0257525 ,
        0.09134978,  0.11247798, -0.04348511, -0.07696104,  0.17563257,
        0.13185213,  0.        , -0.0732034 , -0.1722459 , -0.15415068,
        0.20544397,  0.03509132,  0.03390155,  0.17114826, -0.08803335,
        0.05374428,  0.03428907,  0.11152127,  0.        , -0.07836907,
       -0.12733942, -0.10398971,  0.12838117,  0.02967577,  0.05129329,
        0.06973334, -0.06419316,  0.01098912,  0.17500891,  0.05358425,
        0.05085842, -0.14660347, -0.09006082, -0.10477895,  0.12036368,
        0.0102565 ,  0.        ,  0.18571715, -0.00424629, -0.02586351,
        0.05933944,  0.08288766,  0.02985296, -0.13774193, -0.11620201,
       -0.15890128,  0.11034806,  0.01481509, -0.08167803,  0.22314355,
       -0.0346355 ,  0.0303711 ,  0.12062799,  0.13447791, -0.03025441,
       -0.12334455, -0.12310606, -0.12051602,  0.12051602,  0.05

In [56]:
minMax = MinMaxScaler((0.2,0.8))
usar = minMax.fit_transform(conjunto_train)

In [41]:
# Se usan 6 capas
n_params = 6*window

def coste(params):
    
    coste=0
    
    for i in range(len(conjunto_train)):
        qc = QuantumCircuit(window + 1,1)
        for j in range(window):
            a = conjunto_train[i][j]*np.pi/2
            f = (np.cos(conjunto_train[i][j]*np.pi/2) + 1j*np.sin(conjunto_train[i][j]*np.pi/2))
            matriz = f*np.array( [[ np.cos(a) , -1j*np.sin(a) ],[-1j*np.sin(a), np.cos(a)]])
            op_mat = Operator(matriz)
            qc.append(op_mat,[j+1] )
       
        
    
    return coste;

In [42]:
coste(1)

                
q_0: ───────────
     ┌─────────┐
q_1: ┤ Unitary ├
     ├─────────┤
q_2: ┤ Unitary ├
     └─────────┘
  c: ═══════════
                
                
q_0: ───────────
     ┌─────────┐
q_1: ┤ Unitary ├
     ├─────────┤
q_2: ┤ Unitary ├
     └─────────┘
  c: ═══════════
                
                
q_0: ───────────
     ┌─────────┐
q_1: ┤ Unitary ├
     ├─────────┤
q_2: ┤ Unitary ├
     └─────────┘
  c: ═══════════
                
                
q_0: ───────────
     ┌─────────┐
q_1: ┤ Unitary ├
     ├─────────┤
q_2: ┤ Unitary ├
     └─────────┘
  c: ═══════════
                
                
q_0: ───────────
     ┌─────────┐
q_1: ┤ Unitary ├
     ├─────────┤
q_2: ┤ Unitary ├
     └─────────┘
  c: ═══════════
                
                
q_0: ───────────
     ┌─────────┐
q_1: ┤ Unitary ├
     ├─────────┤
q_2: ┤ Unitary ├
     └─────────┘
  c: ═══════════
                
                
q_0: ───────────
     ┌─────────┐
q_1: ┤ Unitary ├
     ├────────

0

In [26]:
aaaa = np.array([[0,0,0,1],[0,0,1,0],[0,1,0,0],[1,0,0,0]])

In [27]:
aaaa @ aaaa

array([[1, 0, 0, 0],
       [0, 1, 0, 0],
       [0, 0, 1, 0],
       [0, 0, 0, 1]])

In [29]:
np.exp(-1j * np.pi)

(-1-1.2246467991473532e-16j)

In [32]:
np.cos(np.pi) + 1j*np.sin(np.pi)

(-1+1.2246467991473532e-16j)